# Single-channel statistics
Contrast spectral parameters between baseline and encoding, at the single-electrode level

## Set-up

### imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import permutation_test, ttest_rel
from statsmodels.stats.multitest import multipletests

import sys
sys.path.append("../code")
from paths import PROJECT_PATH
from info import *
from stats import mean_difference
from settings import BANDS


### settings

In [2]:
# settings
ALPHA = 0.05 # significance level
FEATURES = ['exponent', 'alpha', 'alpha_adj', 'gamma', 'gamma_adj']

In [3]:
# set plotting style
plt.style.use('../mplstyle/default.mplstyle')

#### functions

In [4]:

def load_stats():
    # load permutatiobn results
    fname = f"{PROJECT_PATH}/data/results/band_power_statistics.csv"
    df_stats = pd.read_csv(fname, index_col=0)
    df_stats = df_stats.loc[df_stats['memory']=='hit']

    # find channels with significant alpha and gamma modulation
    df_stats['sig_all'] = df_stats['alpha_sig'] & df_stats['gamma_sig']
    df_stats['sig_any'] = df_stats['alpha_sig'] | df_stats['gamma_sig']

     #split by conditoin
    df_w = df_stats.loc[df_stats['material']=='words']
    df_f = df_stats.loc[df_stats['material']=='faces']
    
    return df_w, df_f


### load data

In [5]:
# load channel info
chan_info = pd.read_csv(f"{PROJECT_PATH}/data/ieeg_metadata/ieeg_channel_info.csv", index_col=0)
chan_info

,index,patient,chan_idx,label,pos_y,pos_x,pos_z
0,9,pat02,0,A01-A02,-65.43100,61.94490,3.55955
1,10,pat02,1,A02-A03,-70.93895,57.17765,12.15540
2,11,pat02,2,A03-A04,-75.39550,51.39440,20.94335
3,12,pat02,3,A04-A05,-78.91950,43.90980,30.13485
4,13,pat02,4,A05-A06,-80.96735,35.21485,38.13475
...,...,...,...,...,...,...,...
690,1385,pat22,48,MOF_03-MOF_04,36.00000,-14.00000,3.00000
691,1386,pat22,49,MOF_04-MOF_07,40.50000,-14.00000,9.50000
692,1387,pat22,50,MOF_07-MOF_08,44.50000,-14.00000,16.50000
693,1388,pat22,51,MOF_08-MOF_09,48.00000,-14.00000,23.50000


In [6]:
# load single-trial spectral parameters
df_in = pd.read_csv(f"{PROJECT_PATH}/data/results/psd_trial_params.csv")
df = df_in.loc[df_in['memory']=='hit'] # only successful memory trials
df = df.loc[df['ap_mode']=='knee'] # only 'knee' aperiodic mode
df = df.drop(columns=['memory', 'ap_mode'])
df

,patient,trial,chan_idx,material,epoch,offset,exponent,error,r_squared,r2_adj,alpha,alpha_adj,gamma,gamma_adj,knee
576,pat02,0,0,faces,post,7.216933,2.823414,0.403242,0.709088,0.689476,3.784398,-0.320319,2.128099,0.111750,529.607314
577,pat02,0,1,faces,post,6.460279,2.465582,0.412059,0.642264,0.630599,3.458305,-0.193239,1.916144,0.000177,313.521525
578,pat02,0,2,faces,post,7.673479,2.959702,0.393823,0.609863,0.597141,3.711390,0.019705,2.276487,0.067395,8475.967657
579,pat02,0,3,faces,post,7.700448,2.850642,0.380212,0.568571,0.539486,3.712280,0.081378,2.486117,0.063415,10876.147516
580,pat02,0,4,faces,post,7.151837,2.716245,0.395406,0.676009,0.665444,4.055515,0.132710,2.186920,0.041887,1087.373765
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
542699,pat22,57,48,words,pre,5.360639,2.970151,0.435862,0.727372,0.718482,2.261792,-0.099414,-0.167861,-0.059388,-28.168646
542700,pat22,57,49,words,pre,6.240518,3.066793,0.495311,0.668732,0.657930,3.139316,0.002364,0.630701,0.037274,-19.576435
542701,pat22,57,50,words,pre,6.991722,3.426955,0.471361,0.768017,0.760452,2.972699,-0.467061,0.625706,-0.055585,496.603688
542702,pat22,57,51,words,pre,7.138637,3.136275,0.404338,0.802373,0.795929,4.059560,0.070872,1.378580,0.014921,-92.372216


## Main

### Run stats
- paired T-test
- permutation test

In [7]:
# # run paired t-test - combine 'word' and 'face' conditions

# df_p = chan_info.copy()

# # run stats for each channel
# for i_row, row in df_p.iterrows():
#     print(f"Processing {i_row+1}/{len(df_p)}")
#     df_i = df.loc[(df['patient']==row['patient']) & \
#                   (df['chan_idx']==row['chan_idx'])]
    
#     # analyze each spectral feature
#     for feature in FEATURES:
#         # get single-trial features for a given channel
#         values_pre = df_i.loc[df_i['epoch']=='pre', feature].values
#         values_post = df_i.loc[df_i['epoch']=='post', feature].values

#         # run paired t-test
#         stats = ttest_rel(values_pre, values_post, nan_policy='omit')
#         df_p.loc[i_row, f'{feature}_t'] = stats.statistic
#         df_p.loc[i_row, f'{feature}_p'] = stats.pvalue

# # save results
# df_p.to_csv(f"{PROJECT_PATH}/data/results/single_channel_stats.csv")

# df_p

In [29]:
# run paired t-test - split 'word' and 'face' conditions

df_w = chan_info.copy()
df_f = chan_info.copy()

# loop materials
for df_c, material in zip([df_w, df_f], MATERIALS):
    # run stats for each channel
    for i_row, row in df_c.iterrows():
        print(f"Processing {i_row+1}/{len(df_c)}")
        df_i = df.loc[(df['patient']==row['patient']) & \
                      (df['chan_idx']==row['chan_idx']) & \
                      (df['material']==material)]
        
        # analyze each spectral feature
        for feature in FEATURES:
            # get single-trial features for a given channel
            values_pre = df_i.loc[df_i['epoch']=='pre', feature].values
            values_post = df_i.loc[df_i['epoch']=='post', feature].values

            # run paired t-test
            stats = ttest_rel(values_pre, values_post, nan_policy='omit')
            df_c.loc[i_row, f'{feature}_t'] = stats.statistic
            df_c.loc[i_row, f'{feature}_p'] = stats.pvalue

    # save results
    df_c.to_csv(f"{PROJECT_PATH}/data/results/single_channel_stats_{material}.csv", index=False)

df_c

Processing 1/695
Processing 2/695
Processing 3/695
Processing 4/695
Processing 5/695
Processing 6/695
Processing 7/695
Processing 8/695
Processing 9/695
Processing 10/695
Processing 11/695
Processing 12/695
Processing 13/695
Processing 14/695
Processing 15/695
Processing 16/695
Processing 17/695
Processing 18/695
Processing 19/695
Processing 20/695
Processing 21/695
Processing 22/695
Processing 23/695
Processing 24/695
Processing 25/695
Processing 26/695
Processing 27/695
Processing 28/695
Processing 29/695
Processing 30/695
Processing 31/695
Processing 32/695
Processing 33/695
Processing 34/695
Processing 35/695
Processing 36/695
Processing 37/695
Processing 38/695
Processing 39/695
Processing 40/695
Processing 41/695
Processing 42/695
Processing 43/695
Processing 44/695
Processing 45/695
Processing 46/695
Processing 47/695
Processing 48/695
Processing 49/695
Processing 50/695
Processing 51/695
Processing 52/695
Processing 53/695
Processing 54/695
Processing 55/695
Processing 56/695
P

c:\Users\micha\anaconda3\envs\oscillation_vs_exponent\Lib\site-packages\scipy\stats\_stats_py.py:1103: RuntimeWarning: divide by zero encountered in divide
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero
c:\Users\micha\anaconda3\envs\oscillation_vs_exponent\Lib\site-packages\scipy\stats\_stats_py.py:1103: RuntimeWarning: invalid value encountered in scalar multiply
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero


Processing 305/695
Processing 306/695
Processing 307/695
Processing 308/695
Processing 309/695
Processing 310/695
Processing 311/695
Processing 312/695
Processing 313/695
Processing 314/695
Processing 315/695
Processing 316/695
Processing 317/695
Processing 318/695
Processing 319/695
Processing 320/695
Processing 321/695
Processing 322/695
Processing 323/695
Processing 324/695
Processing 325/695
Processing 326/695
Processing 327/695
Processing 328/695
Processing 329/695
Processing 330/695
Processing 331/695
Processing 332/695
Processing 333/695
Processing 334/695
Processing 335/695
Processing 336/695
Processing 337/695
Processing 338/695
Processing 339/695
Processing 340/695
Processing 341/695
Processing 342/695
Processing 343/695
Processing 344/695
Processing 345/695
Processing 346/695
Processing 347/695
Processing 348/695
Processing 349/695
Processing 350/695
Processing 351/695
Processing 352/695
Processing 353/695
Processing 354/695
Processing 355/695
Processing 356/695
Processing 3

,index,patient,chan_idx,label,pos_y,pos_x,pos_z,exponent_t,exponent_p,alpha_t,alpha_p,alpha_adj_t,alpha_adj_p,gamma_t,gamma_p,gamma_adj_t,gamma_adj_p
0,9,pat02,0,A01-A02,-65.43100,61.94490,3.55955,3.057932,0.003638,2.684405,9.943088e-03,0.506743,0.614655,-1.084905,0.283383,-1.776797,0.081939
1,10,pat02,1,A02-A03,-70.93895,57.17765,12.15540,2.264746,0.027724,5.853191,3.280561e-07,0.862056,0.392615,1.258038,0.213998,0.572354,0.569549
2,11,pat02,2,A03-A04,-75.39550,51.39440,20.94335,3.998906,0.000177,7.949677,5.971463e-11,4.243775,0.000077,-2.003634,0.049631,-1.301115,0.198194
3,12,pat02,3,A04-A05,-78.91950,43.90980,30.13485,1.437197,0.155857,0.676885,5.010804e-01,0.314873,0.753950,0.319636,0.750354,1.048645,0.298547
4,13,pat02,4,A05-A06,-80.96735,35.21485,38.13475,1.638490,0.106554,2.831851,6.290720e-03,0.586686,0.559616,2.408278,0.019117,0.882954,0.380785
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
690,1385,pat22,48,MOF_03-MOF_04,36.00000,-14.00000,3.00000,0.572336,0.569659,-0.133970,8.939644e-01,0.405102,0.687130,0.100416,0.920416,0.967543,0.337930
691,1386,pat22,49,MOF_04-MOF_07,40.50000,-14.00000,9.50000,0.964981,0.339021,-0.014713,9.883171e-01,1.235656,0.222140,-0.399498,0.691162,-1.265464,0.211346
692,1387,pat22,50,MOF_07-MOF_08,44.50000,-14.00000,16.50000,1.543046,0.128770,-1.120303,2.676358e-01,-0.994061,0.324710,0.427619,0.670661,-1.720093,0.091250
693,1388,pat22,51,MOF_08-MOF_09,48.00000,-14.00000,23.50000,1.708824,0.093447,-1.597690,1.161717e-01,0.839976,0.404767,-1.780310,0.080868,-0.409098,0.684148


In [8]:
# # run permutation test - combine 'word' and 'face' conditions

# N_ITER = 1000

# df_p = chan_info.copy()

# for i_row, row in df_p.iterrows():
#     print(f"Processing {i_row+1}/{len(df_p)}")
#     df_i = df.loc[(df['patient']==row['patient']) & \
#                   (df['chan_idx']==row['chan_idx'])]
#     for feature in FEATURES:
#         values_pre = df_i.loc[df_i['epoch']=='pre', feature].values
#         values_post = df_i.loc[df_i['epoch']=='post', feature].values

#         stats = permutation_test([values_pre, values_post], 
#                             statistic=mean_difference, 
#                             permutation_type='samples',
#                             n_resamples=N_ITER,
#                             alternative='two-sided',
#                             random_state=0)
#         df_p.loc[i_row, f'{feature}_t'] = stats.statistic
#         df_p.loc[i_row, f'{feature}_p'] = stats.pvalue

# save results
# df_p.to_csv(f"{PROJECT_PATH}/data/results/single_channel_permutation.csv")

# df_p

In [47]:
# run permutation test - split 'word' and 'face' conditions

N_ITER = 1000

df_w = chan_info.copy()
df_f = chan_info.copy()

# loop materials
for df_c, material in zip([df_w, df_f], MATERIALS):
    for i_row, row in df_c.iterrows():
        print(f"Processing {i_row+1}/{len(df_c)}")
        df_i = df.loc[(df['patient']==row['patient']) & \
                    (df['chan_idx']==row['chan_idx'])]
        for feature in FEATURES:
            values_pre = df_i.loc[df_i['epoch']=='pre', feature].values
            values_post = df_i.loc[df_i['epoch']=='post', feature].values
            stats = permutation_test([values_pre, values_post], 
                                statistic=mean_difference, 
                                permutation_type='samples',
                                n_resamples=N_ITER,
                                alternative='two-sided',
                                random_state=0)
            df_c.loc[i_row, f'{feature}_t'] = stats.statistic
            df_c.loc[i_row, f'{feature}_p'] = stats.pvalue

    # save results
    df_c.to_csv(f"{PROJECT_PATH}/data/results/single_channel_permutation_{material}.csv", index=False)

    df_c

Processing 1/695
Processing 2/695
Processing 3/695
Processing 4/695
Processing 5/695
Processing 6/695
Processing 7/695
Processing 8/695
Processing 9/695
Processing 10/695
Processing 11/695
Processing 12/695
Processing 13/695
Processing 14/695
Processing 15/695
Processing 16/695
Processing 17/695
Processing 18/695
Processing 19/695
Processing 20/695
Processing 21/695
Processing 22/695
Processing 23/695
Processing 24/695
Processing 25/695
Processing 26/695
Processing 27/695
Processing 28/695
Processing 29/695
Processing 30/695
Processing 31/695
Processing 32/695
Processing 33/695
Processing 34/695
Processing 35/695
Processing 36/695
Processing 37/695
Processing 38/695
Processing 39/695
Processing 40/695
Processing 41/695
Processing 42/695
Processing 43/695
Processing 44/695
Processing 45/695
Processing 46/695
Processing 47/695
Processing 48/695
Processing 49/695
Processing 50/695
Processing 51/695
Processing 52/695
Processing 53/695
Processing 54/695
Processing 55/695
Processing 56/695
P

### analyze results
count number of significant channels

In [7]:
# load results from above
# df_p = pd.read_csv(f"{PROJECT_PATH}/data/results/single_channel_permutation.csv", index_col=0)
df_p = pd.read_csv(f"{PROJECT_PATH}/data/results/single_channel_stats.csv", index_col=0)
df_p

,index,patient,chan_idx,label,pos_y,pos_x,pos_z,exponent_t,exponent_p,alpha_t,alpha_p,alpha_adj_t,alpha_adj_p,gamma_t,gamma_p,gamma_adj_t,gamma_adj_p
0,9,pat02,0,A01-A02,-65.43100,61.94490,3.55955,2.835601,5.436901e-03,1.676486,9.645729e-02,3.030348,0.003045,0.404330,6.867484e-01,-0.119329,0.905230
1,10,pat02,1,A02-A03,-70.93895,57.17765,12.15540,3.645792,3.991216e-04,3.047198,2.855607e-03,2.396835,0.018121,0.965618,3.362269e-01,-0.416101,0.678098
2,11,pat02,2,A03-A04,-75.39550,51.39440,20.94335,3.741851,2.685186e-04,6.055645,1.287034e-08,4.059815,0.000083,-3.266588,1.377849e-03,-2.813170,0.005633
3,12,pat02,3,A04-A05,-78.91950,43.90980,30.13485,3.291578,1.273745e-03,2.215688,2.840065e-02,1.180514,0.239887,-0.451134,6.526225e-01,0.378919,0.705348
4,13,pat02,4,A05-A06,-80.96735,35.21485,38.13475,2.714708,7.513506e-03,4.745391,5.297972e-06,3.722206,0.000292,1.569405,1.189297e-01,1.217230,0.225672
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
690,1385,pat22,48,MOF_03-MOF_04,36.00000,-14.00000,3.00000,0.251394,8.019919e-01,-0.302382,7.629481e-01,-2.328770,0.021768,0.866149,3.883465e-01,0.787370,0.432806
691,1386,pat22,49,MOF_04-MOF_07,40.50000,-14.00000,9.50000,0.549385,5.838758e-01,2.187215,3.088321e-02,1.617989,0.108723,1.395341,1.657756e-01,1.415666,0.159751
692,1387,pat22,50,MOF_07-MOF_08,44.50000,-14.00000,16.50000,1.740700,8.455814e-02,0.433874,6.652372e-01,0.931518,0.353764,1.634162,1.051099e-01,1.466778,0.145316
693,1388,pat22,51,MOF_08-MOF_09,48.00000,-14.00000,23.50000,3.953345,1.372780e-04,-0.755971,4.512970e-01,0.321435,0.748514,-5.813815,6.170116e-08,-5.145211,0.000001


In [10]:
# # print minimum p-values
# print("Minimum p-values:")
# for feature in FEATURES:
#     p_values = df_p[f'{feature}_p'].values
#     print(f"    {feature}: \t{np.nanmin(p_values):0.1e}")

In [8]:
# count the number of channels with a significant difference (increase and 
# decrease seperately) for each feature - across all channels

# count
for feature in FEATURES:
    df_p[f'{feature}_sig'] = 0
    df_p.loc[(df_p[f'{feature}_p']<0.05) & (df_p[f'{feature}_t']>0), f'{feature}_sig'] = -1 # ! this is different for permutation test and t-test (sign of t-statistic) !
    df_p.loc[(df_p[f'{feature}_p']<0.05) & (df_p[f'{feature}_t']<0), f'{feature}_sig'] = 1 # ! this is different for permutation test and t-test (sign of t-statistic) !

# print
for feature in FEATURES:
    print(f"\n{feature}:")
    print(f"\tIncrease: {df_p.loc[df_p[f'{feature}_sig']==1].shape[0]} ({df_p.loc[df_p[f'{feature}_sig']==1].shape[0]/df_p.shape[0]*100:.2f}%)")
    print(f"\tDecrease: {df_p.loc[df_p[f'{feature}_sig']==-1].shape[0]} ({df_p.loc[df_p[f'{feature}_sig']==-1].shape[0]/df_p.shape[0]*100:.2f}%)")
    print(f"\tNo change: {df_p.loc[df_p[f'{feature}_sig']==0].shape[0]} ({df_p.loc[df_p[f'{feature}_sig']==0].shape[0]/df_p.shape[0]*100:.2f}%)")


exponent:
	Increase: 32 (4.60%)
	Decrease: 210 (30.22%)
	No change: 453 (65.18%)

alpha:
	Increase: 135 (19.42%)
	Decrease: 144 (20.72%)
	No change: 416 (59.86%)

alpha_adj:
	Increase: 93 (13.38%)
	Decrease: 135 (19.42%)
	No change: 467 (67.19%)

gamma:
	Increase: 181 (26.04%)
	Decrease: 121 (17.41%)
	No change: 393 (56.55%)

gamma_adj:
	Increase: 150 (21.58%)
	Decrease: 92 (13.24%)
	No change: 453 (65.18%)


In [9]:
# count number of task-modulated channels with significant effects

# reduce to task-modulated channels
df_mod = pd.read_csv(f"{PROJECT_PATH}/data/results/ieeg_modulated_channels.csv", 
                     index_col=0)
df_sig = df_p.loc[df_mod['sig_all']]
print(f"N task-modulated channels: {df_sig.shape[0]}")

# print counts
for feature in FEATURES:
    print(f"\n{feature}:")
    print(f"\tIncrease: {df_sig.loc[df_sig[f'{feature}_sig']==1].shape[0]} ({df_sig.loc[df_sig[f'{feature}_sig']==1].shape[0]/df_sig.shape[0]*100:.2f}%)")
    print(f"\tDecrease: {df_sig.loc[df_sig[f'{feature}_sig']==-1].shape[0]} ({df_sig.loc[df_sig[f'{feature}_sig']==-1].shape[0]/df_sig.shape[0]*100:.2f}%)")
    print(f"\tNo change: {df_sig.loc[df_sig[f'{feature}_sig']==0].shape[0]} ({df_sig.loc[df_sig[f'{feature}_sig']==0].shape[0]/df_sig.shape[0]*100:.2f}%)")

N task-modulated channels: 39

exponent:
	Increase: 0 (0.00%)
	Decrease: 24 (61.54%)
	No change: 15 (38.46%)

alpha:
	Increase: 8 (20.51%)
	Decrease: 29 (74.36%)
	No change: 2 (5.13%)

alpha_adj:
	Increase: 5 (12.82%)
	Decrease: 28 (71.79%)
	No change: 6 (15.38%)

gamma:
	Increase: 28 (71.79%)
	Decrease: 8 (20.51%)
	No change: 3 (7.69%)

gamma_adj:
	Increase: 28 (71.79%)
	Decrease: 8 (20.51%)
	No change: 3 (7.69%)


### Correct for multiple comparison

In [13]:
# # show in simulation the expected FPR for permutation test

# N_ITER = 1000 # number of iterations
# N = 100 # number of samples

# # run permutation test on random data
# p_values = np.zeros(N_ITER)
# for i in range(N_ITER):
#     data_0 = np.random.normal(0, 1, N)
#     data_1 = np.random.normal(0, 1, N)
#     p_values[i] = permutation_test([data_0, data_1], 
#                                     statistic=mean_difference, 
#                                     permutation_type='samples',
#                                     n_resamples=1000,
#                                     alternative='two-sided',
#                                     random_state=0).pvalue

# # plot hist of p-values
# fig, ax = plt.subplots(1,1, figsize=[6,4])
# ax.hist(p_values, bins=20)
# ax.set(xlabel="p-value", ylabel="count")
# ax.set_title("Permutation test FPR")
# plt.show()

# # print proportion of false positive
# print(f"Expected FPR: {np.sum(p_values<0.05)/N_ITER}")


In [10]:
# compute corrected pvalues - all channels

# correct p-values
for feature in FEATURES:
    results = multipletests(df_p[f"{feature}_p"], alpha=0.05, method='holm')
    df_p.insert(df_p.shape[1], f"{feature}_pval_corr", results[1])

# determine significance and direction of effect
for feature in FEATURES:
    df_p[f'{feature}_sig_corr'] = 0
    df_p.loc[(df_p[f'{feature}_pval_corr']<0.05) & (df_p[f'{feature}_t']>0), f'{feature}_sig_corr'] = -1 # ! this is different for permutation test and t-test (sign of t-statistic) !
    df_p.loc[(df_p[f'{feature}_pval_corr']<0.05) & (df_p[f'{feature}_t']<0), f'{feature}_sig_corr'] = 1 # ! this is different for permutation test and t-test (sign of t-statistic) !

# print number of significant channels
for feature in FEATURES:
    print(f"\n{feature}:")
    print(f"\tIncrease: {df_p.loc[df_p[f'{feature}_sig_corr']==1].shape[0]} ({df_p.loc[df_p[f'{feature}_sig_corr']==1].shape[0]/df_p.shape[0]*100:.2f}%)")
    print(f"\tDecrease: {df_p.loc[df_p[f'{feature}_sig_corr']==-1].shape[0]} ({df_p.loc[df_p[f'{feature}_sig_corr']==-1].shape[0]/df_p.shape[0]*100:.2f}%)")
    print(f"\tNo change: {df_p.loc[df_p[f'{feature}_sig_corr']==0].shape[0]} ({df_p.loc[df_p[f'{feature}_sig_corr']==0].shape[0]/df_p.shape[0]*100:.2f}%)")



exponent:
	Increase: 1 (0.14%)
	Decrease: 65 (9.35%)
	No change: 629 (90.50%)

alpha:
	Increase: 24 (3.45%)
	Decrease: 64 (9.21%)
	No change: 607 (87.34%)

alpha_adj:
	Increase: 11 (1.58%)
	Decrease: 50 (7.19%)
	No change: 634 (91.22%)

gamma:
	Increase: 76 (10.94%)
	Decrease: 21 (3.02%)
	No change: 598 (86.04%)

gamma_adj:
	Increase: 62 (8.92%)
	Decrease: 11 (1.58%)
	No change: 622 (89.50%)


In [11]:
# compute corrected pvalues - task-modulated channels

# correct p-values
for feature in FEATURES:
    results = multipletests(df_sig[f"{feature}_p"], alpha=0.05, method='holm')
    df_sig.insert(df_sig.shape[1], f"{feature}_pval_corr", results[1])

# determine significance and direction of effect
for feature in FEATURES:
    df_sig.insert(df_sig.shape[1], f'{feature}_sig_corr', 0)
    df_sig.loc[(df_sig[f'{feature}_pval_corr']<0.05) & (df_sig[f'{feature}_t']>0), f'{feature}_sig_corr'] = -1 # ! this is different for permutation test and t-test (sign of t-statistic) !
    df_sig.loc[(df_sig[f'{feature}_pval_corr']<0.05) & (df_sig[f'{feature}_t']<0), f'{feature}_sig_corr'] = 1 # ! this is different for permutation test and t-test (sign of t-statistic) !

# print number of significant channels
for feature in FEATURES:
    print(f"\n{feature}:")
    print(f"\tIncrease: {df_sig.loc[df_sig[f'{feature}_sig_corr']==1].shape[0]} ({df_sig.loc[df_sig[f'{feature}_sig_corr']==1].shape[0]/df_sig.shape[0]*100:.2f}%)")
    print(f"\tDecrease: {df_sig.loc[df_sig[f'{feature}_sig_corr']==-1].shape[0]} ({df_sig.loc[df_sig[f'{feature}_sig_corr']==-1].shape[0]/df_sig.shape[0]*100:.2f}%)")
    print(f"\tNo change: {df_sig.loc[df_sig[f'{feature}_sig_corr']==0].shape[0]} ({df_sig.loc[df_sig[f'{feature}_sig_corr']==0].shape[0]/df_sig.shape[0]*100:.2f}%)")


exponent:
	Increase: 0 (0.00%)
	Decrease: 16 (41.03%)
	No change: 23 (58.97%)

alpha:
	Increase: 7 (17.95%)
	Decrease: 29 (74.36%)
	No change: 3 (7.69%)

alpha_adj:
	Increase: 1 (2.56%)
	Decrease: 25 (64.10%)
	No change: 13 (33.33%)

gamma:
	Increase: 28 (71.79%)
	Decrease: 8 (20.51%)
	No change: 3 (7.69%)

gamma_adj:
	Increase: 28 (71.79%)
	Decrease: 5 (12.82%)
	No change: 6 (15.38%)


In [13]:
# save results
df_sig.to_csv(f"{PROJECT_PATH}/data/ieeg_stats/single_channel_stats_corr.csv")

#### split by material

In [47]:
df_w, df_f = load_stats()

for df in [df_w, df_f]:
    df.drop(columns=['alpha_pre', 'alpha_post', 'alpha_pval', 'alpha_sign', 
                     'gamma_pre', 'gamma_post', 'gamma_pval', 'gamma_sign',
                     'alpha_sig', 'gamma_sig', 'sig_any'], inplace=True)
    
df_w

,patient,material,memory,chan_idx,sig_all
18,pat02,words,hit,0,False
19,pat02,words,hit,1,False
20,pat02,words,hit,2,False
21,pat02,words,hit,3,False
22,pat02,words,hit,4,False
...,...,...,...,...,...
2722,pat22,words,hit,48,False
2723,pat22,words,hit,49,False
2724,pat22,words,hit,50,False
2725,pat22,words,hit,51,True


In [48]:
# load
# df_wst = pd.read_csv(f"{PROJECT_PATH}/data/results/single_channel_permutation_words.csv", index_col=0)
# df_fst = pd.read_csv(f"{PROJECT_PATH}/data/results/single_channel_permutation_faces.csv", index_col=0)
df_wst = pd.read_csv(f"{PROJECT_PATH}/data/results/single_channel_stats_words.csv", index_col=0)
df_fst = pd.read_csv(f"{PROJECT_PATH}/data/results/single_channel_stats_faces.csv", index_col=0)

# merge
df_w = df_wst.merge(df_w, on=['patient','chan_idx'])
df_f = df_fst.merge(df_f, on=['patient','chan_idx'])

# drop non-task-modulated channels
df_w = df_w.loc[df_w['sig_all']].drop(columns=['sig_all'])
df_f = df_f.loc[df_f['sig_all']].drop(columns=['sig_all'])

# show
print(len(df_w), len(df_f))
df_w

104 97


,index,patient,chan_idx,label,pos_y,pos_x,pos_z,exponent_t,exponent_p,alpha_t,alpha_p,alpha_adj_t,alpha_adj_p,gamma_t,gamma_p,gamma_adj_t,gamma_adj_p,material,memory
9,100,pat04,0,A01-A09,-58.30120,52.43075,-26.23685,4.032624,2.107200e-04,6.770440,2.233319e-08,0.671490,0.505341,-8.912848,1.682913e-11,-1.907723,0.062821,words,hit
54,145,pat04,45,A01-A02,-63.34330,53.12490,-21.48615,5.868628,4.555194e-07,7.522039,1.510788e-09,0.168096,0.867245,-6.706357,2.515065e-08,-0.123916,0.901921,words,hit
58,149,pat04,49,A10-A11,-52.12025,61.31445,-15.30360,0.523225,6.031756e-01,1.630562,1.093948e-01,2.450568,0.017880,-1.450528,1.532829e-01,0.796636,0.429505,words,hit
67,158,pat04,58,A28-A29,-26.97260,66.43845,5.20775,-1.827064,7.332320e-02,1.439572,1.558691e-01,0.685729,0.495873,2.614423,1.161130e-02,0.534061,0.595532,words,hit
88,179,pat04,79,A60-A62,16.19335,56.90075,1.58265,1.386216,1.723639e-01,1.377683,1.749672e-01,-0.580096,0.564681,1.428293,1.599625e-01,-0.306261,0.760788,words,hit
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
686,1381,pat22,44,LOF_08-LOF_09,47.00000,-25.00000,24.00000,1.914847,6.062523e-02,1.336771,1.867030e-01,0.655598,0.514767,-4.709344,1.679407e-05,1.287626,0.203172,words,hit
688,1383,pat22,46,MOF_01-MOF_02,28.00000,-14.00000,-11.50000,0.060266,9.521586e-01,-0.811777,4.203569e-01,2.296161,0.025431,1.534736,1.304796e-01,-0.454518,0.651213,words,hit
689,1384,pat22,47,MOF_02-MOF_03,31.50000,-14.00000,-4.00000,1.026219,3.092008e-01,-1.514964,1.354059e-01,0.288600,0.773953,2.564863,1.302896e-02,1.182913,0.241841,words,hit
693,1388,pat22,51,MOF_08-MOF_09,48.00000,-14.00000,23.50000,3.753384,4.164932e-04,0.383175,7.030407e-01,0.223396,0.824040,-6.713844,1.026949e-08,-0.146502,0.884052,words,hit


In [49]:
# compute corrected pvalues - task-modulated channels

for df_sig, material in zip([df_w, df_f], ['words', 'faces']):
    print(f"\n\n{material}-encoding ({len(df_sig)} channels):")
    # correct p-values
    for feature in FEATURES:
        results = multipletests(df_sig[f"{feature}_p"], alpha=0.05, method='holm')
        df_sig.insert(df_sig.shape[1], f"{feature}_pval_corr", results[1])

    # determine significance and direction of effect
    for feature in FEATURES:
        df_sig.insert(df_sig.shape[1], f'{feature}_sig_corr', 0)
        df_sig.loc[(df_sig[f'{feature}_pval_corr']<0.05) & (df_sig[f'{feature}_t']>0), f'{feature}_sig_corr'] = -1 # ! this is different for permutation test and t-test (sign of t-statistic) !
        df_sig.loc[(df_sig[f'{feature}_pval_corr']<0.05) & (df_sig[f'{feature}_t']<0), f'{feature}_sig_corr'] = 1 # ! this is different for permutation test and t-test (sign of t-statistic) !

    # print number of significant channels
    for feature in FEATURES:
        print(f"\n{feature}:")
        print(f"\tIncrease: {df_sig.loc[df_sig[f'{feature}_sig_corr']==1].shape[0]} ({df_sig.loc[df_sig[f'{feature}_sig_corr']==1].shape[0]/df_sig.shape[0]*100:.2f}%)")
        print(f"\tDecrease: {df_sig.loc[df_sig[f'{feature}_sig_corr']==-1].shape[0]} ({df_sig.loc[df_sig[f'{feature}_sig_corr']==-1].shape[0]/df_sig.shape[0]*100:.2f}%)")
        print(f"\tNo change: {df_sig.loc[df_sig[f'{feature}_sig_corr']==0].shape[0]} ({df_sig.loc[df_sig[f'{feature}_sig_corr']==0].shape[0]/df_sig.shape[0]*100:.2f}%)")



words-encoding (104 channels):

exponent:
	Increase: 0 (0.00%)
	Decrease: 22 (21.15%)
	No change: 82 (78.85%)

alpha:
	Increase: 6 (5.77%)
	Decrease: 33 (31.73%)
	No change: 65 (62.50%)

alpha_adj:
	Increase: 0 (0.00%)
	Decrease: 5 (4.81%)
	No change: 99 (95.19%)

gamma:
	Increase: 37 (35.58%)
	Decrease: 4 (3.85%)
	No change: 63 (60.58%)

gamma_adj:
	Increase: 0 (0.00%)
	Decrease: 0 (0.00%)
	No change: 104 (100.00%)


faces-encoding (97 channels):

exponent:
	Increase: 2 (2.06%)
	Decrease: 26 (26.80%)
	No change: 69 (71.13%)

alpha:
	Increase: 9 (9.28%)
	Decrease: 30 (30.93%)
	No change: 58 (59.79%)

alpha_adj:
	Increase: 3 (3.09%)
	Decrease: 4 (4.12%)
	No change: 90 (92.78%)

gamma:
	Increase: 39 (40.21%)
	Decrease: 5 (5.15%)
	No change: 53 (54.64%)

gamma_adj:
	Increase: 0 (0.00%)
	Decrease: 0 (0.00%)
	No change: 97 (100.00%)


In [41]:
# compute corrected pvalues - alpha-modulated channels

# load task-modulation info
df_w, df_f = load_stats()
for df in [df_w, df_f]:
    df.drop(columns=['alpha_pre', 'alpha_post', 'alpha_pval', 'alpha_sign', 
                     'gamma_pre', 'gamma_post', 'gamma_pval', 'gamma_sign'], 
                     inplace=True)
    
# load single-channel stats
df_wst = pd.read_csv(f"{PROJECT_PATH}/data/results/single_channel_stats_words.csv", index_col=0)
df_fst = pd.read_csv(f"{PROJECT_PATH}/data/results/single_channel_stats_faces.csv", index_col=0)

# merge
df_w = df_wst.merge(df_w, on=['patient','chan_idx'])
df_f = df_fst.merge(df_f, on=['patient','chan_idx'])

# drop non-task-modulated channels
df_w = df_w.loc[df_w['alpha_sig']].drop(columns=['alpha_sig'])
df_f = df_f.loc[df_f['alpha_sig']].drop(columns=['alpha_sig'])

# compute corrected pvalues 
for df_sig, material in zip([df_w, df_f], ['words', 'faces']):
    print(f"\n\n{material}-encoding")
    # correct p-values
    for feature in FEATURES:
        results = multipletests(df_sig[f"{feature}_p"], alpha=0.05, method='holm')
        df_sig.insert(df_sig.shape[1], f"{feature}_pval_corr", results[1])

    # determine significance and direction of effect
    for feature in FEATURES:
        df_sig.insert(df_sig.shape[1], f'{feature}_sig_corr', 0)
        df_sig.loc[(df_sig[f'{feature}_pval_corr']<0.05) & (df_sig[f'{feature}_t']>0), f'{feature}_sig_corr'] = -1 # ! this is different for permutation test and t-test (sign of t-statistic) !
        df_sig.loc[(df_sig[f'{feature}_pval_corr']<0.05) & (df_sig[f'{feature}_t']<0), f'{feature}_sig_corr'] = 1 # ! this is different for permutation test and t-test (sign of t-statistic) !

    # print number of significant channels
    for feature in FEATURES:
        print(f"\n{feature}:")
        print(f"\tIncrease: {df_sig.loc[df_sig[f'{feature}_sig_corr']==1].shape[0]} ({df_sig.loc[df_sig[f'{feature}_sig_corr']==1].shape[0]/df_sig.shape[0]*100:.2f}%)")
        print(f"\tDecrease: {df_sig.loc[df_sig[f'{feature}_sig_corr']==-1].shape[0]} ({df_sig.loc[df_sig[f'{feature}_sig_corr']==-1].shape[0]/df_sig.shape[0]*100:.2f}%)")
        print(f"\tNo change: {df_sig.loc[df_sig[f'{feature}_sig_corr']==0].shape[0]} ({df_sig.loc[df_sig[f'{feature}_sig_corr']==0].shape[0]/df_sig.shape[0]*100:.2f}%)")
    



words-encoding

exponent:
	Increase: 0 (0.00%)
	Decrease: 22 (8.87%)
	No change: 226 (91.13%)

alpha:
	Increase: 9 (3.63%)
	Decrease: 43 (17.34%)
	No change: 196 (79.03%)

alpha_adj:
	Increase: 0 (0.00%)
	Decrease: 6 (2.42%)
	No change: 242 (97.58%)

gamma:
	Increase: 31 (12.50%)
	Decrease: 4 (1.61%)
	No change: 213 (85.89%)

gamma_adj:
	Increase: 0 (0.00%)
	Decrease: 0 (0.00%)
	No change: 248 (100.00%)


faces-encoding

exponent:
	Increase: 4 (1.70%)
	Decrease: 30 (12.77%)
	No change: 201 (85.53%)

alpha:
	Increase: 16 (6.81%)
	Decrease: 48 (20.43%)
	No change: 171 (72.77%)

alpha_adj:
	Increase: 3 (1.28%)
	Decrease: 4 (1.70%)
	No change: 228 (97.02%)

gamma:
	Increase: 34 (14.47%)
	Decrease: 1 (0.43%)
	No change: 200 (85.11%)

gamma_adj:
	Increase: 0 (0.00%)
	Decrease: 0 (0.00%)
	No change: 235 (100.00%)


In [43]:
# compute corrected pvalues - gamma-modulated channels

# load task-modulation info
df_w, df_f = load_stats()
for df in [df_w, df_f]:
    df.drop(columns=['alpha_pre', 'alpha_post', 'alpha_pval', 'alpha_sign', 
                     'gamma_pre', 'gamma_post', 'gamma_pval', 'gamma_sign'], 
                     inplace=True)
    
# load single-channel stats
df_wst = pd.read_csv(f"{PROJECT_PATH}/data/results/single_channel_stats_words.csv", index_col=0)
df_fst = pd.read_csv(f"{PROJECT_PATH}/data/results/single_channel_stats_faces.csv", index_col=0)

# merge
df_w = df_wst.merge(df_w, on=['patient','chan_idx'])
df_f = df_fst.merge(df_f, on=['patient','chan_idx'])

# drop non-task-modulated channels
df_w = df_w.loc[df_w['gamma_sig']].drop(columns=['gamma_sig'])
df_f = df_f.loc[df_f['gamma_sig']].drop(columns=['gamma_sig'])

# compute corrected pvalues 
for df_sig, material in zip([df_w, df_f], ['words', 'faces']):
    print(f"\n\n{material}-encoding")
    # correct p-values
    for feature in FEATURES:
        results = multipletests(df_sig[f"{feature}_p"], alpha=0.05, method='holm')
        df_sig.insert(df_sig.shape[1], f"{feature}_pval_corr", results[1])

    # determine significance and direction of effect
    for feature in FEATURES:
        df_sig.insert(df_sig.shape[1], f'{feature}_sig_corr', 0)
        df_sig.loc[(df_sig[f'{feature}_pval_corr']<0.05) & (df_sig[f'{feature}_t']>0), f'{feature}_sig_corr'] = -1 # ! this is different for permutation test and t-test (sign of t-statistic) !
        df_sig.loc[(df_sig[f'{feature}_pval_corr']<0.05) & (df_sig[f'{feature}_t']<0), f'{feature}_sig_corr'] = 1 # ! this is different for permutation test and t-test (sign of t-statistic) !

    # print number of significant channels
    for feature in FEATURES:
        print(f"\n{feature}:")
        print(f"\tIncrease: {df_sig.loc[df_sig[f'{feature}_sig_corr']==1].shape[0]} ({df_sig.loc[df_sig[f'{feature}_sig_corr']==1].shape[0]/df_sig.shape[0]*100:.2f}%)")
        print(f"\tDecrease: {df_sig.loc[df_sig[f'{feature}_sig_corr']==-1].shape[0]} ({df_sig.loc[df_sig[f'{feature}_sig_corr']==-1].shape[0]/df_sig.shape[0]*100:.2f}%)")
        print(f"\tNo change: {df_sig.loc[df_sig[f'{feature}_sig_corr']==0].shape[0]} ({df_sig.loc[df_sig[f'{feature}_sig_corr']==0].shape[0]/df_sig.shape[0]*100:.2f}%)")
    



words-encoding

exponent:
	Increase: 0 (0.00%)
	Decrease: 26 (10.92%)
	No change: 212 (89.08%)

alpha:
	Increase: 3 (1.26%)
	Decrease: 26 (10.92%)
	No change: 209 (87.82%)

alpha_adj:
	Increase: 0 (0.00%)
	Decrease: 3 (1.26%)
	No change: 235 (98.74%)

gamma:
	Increase: 57 (23.95%)
	Decrease: 12 (5.04%)
	No change: 169 (71.01%)

gamma_adj:
	Increase: 0 (0.00%)
	Decrease: 0 (0.00%)
	No change: 238 (100.00%)


faces-encoding

exponent:
	Increase: 1 (0.45%)
	Decrease: 32 (14.48%)
	No change: 188 (85.07%)

alpha:
	Increase: 6 (2.71%)
	Decrease: 26 (11.76%)
	No change: 189 (85.52%)

alpha_adj:
	Increase: 2 (0.90%)
	Decrease: 3 (1.36%)
	No change: 216 (97.74%)

gamma:
	Increase: 53 (23.98%)
	Decrease: 8 (3.62%)
	No change: 160 (72.40%)

gamma_adj:
	Increase: 0 (0.00%)
	Decrease: 0 (0.00%)
	No change: 221 (100.00%)
